In [1]:
import tensorflow.keras
tensorflow.keras.backend.set_learning_phase(0)
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.framework import convert_to_constants
import tensorflow_addons as tfa
from tensorflow_addons.image import resampler
import os
import cv2
import matplotlib.pyplot as plt

tfa.image.resampler

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import numpy as np
model_dir = "./config/hfnet_v2"
output_saved_model_dir = "./config/hfnet_v2_trt" 
output_saved_model_dir_netvlad = "./config/hfnet_v2_netvlad_trt" 
output_frozen_path = "./config/hf_frozen_208x400_v2.pb"
output_frozen_path_sp = "./config/hf_frozen_208x400_v2_superpoint.pb"
output_frozen_path_netvlad = "./config/hf_frozen_208x400_netvlad.pb"
verify_images_path= "/ssd/drone_data_process/images"


img = cv2.imread("./db1.jpg")
img = cv2.resize(img, (400, 400))
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img[0:208, 0:400]
tf.test.is_gpu_available()

tfa.register.register_all()

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (400, 208))
        if img is not None:
            images.append(img)
    return images
images = load_images_from_folder(verify_images_path)
print(f"Load {len(images)} images")

1 Physical GPUs, 1 Logical GPUs
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Load 5735 images


In [2]:
class Net(object):

    def __init__(self, model_path):
        self.model_filepath = model_path
        self.load_graph(model_filepath=self.model_filepath)


    def load_graph(self, model_filepath):
        print("Loading model...")


        with tf.io.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())

        with tf.compat.v1.Session() as sess:
            with tf.Graph().as_default() as graph:
                tf.import_graph_def(graph_def, name='')
                signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
                    inputs={
                        'image':graph.get_tensor_by_name('image:0'),
                        'k':graph.get_tensor_by_name('k:0'),
                        'radius':graph.get_tensor_by_name('radius:0')
                           },
                    outputs={'global_descriptor': graph.get_tensor_by_name('global_descriptor:0'),
                        'keypoints': graph.get_tensor_by_name('keypoints:0'),
                        'local_descriptors': graph.get_tensor_by_name('local_descriptors:0')}
                )
                builder = tf.compat.v1.saved_model.Builder("./config/hfnet_v2/")
                builder.add_meta_graph_and_variables(
                    sess=sess,
                    tags=[tf.compat.v1.saved_model.tag_constants.SERVING],
                    signature_def_map={'serving_default': signature}
                )
                builder.save()
class NetVLAD(object):

    def __init__(self, model_path):
        self.model_filepath = model_path
        self.load_graph(model_filepath=self.model_filepath)


    def load_graph(self, model_filepath):
        print("Loading model...")


        with tf.io.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())

        with tf.compat.v1.Session() as sess:
            with tf.Graph().as_default() as graph:
                tf.import_graph_def(graph_def, name='')
                signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
                    inputs={
                        'image':graph.get_tensor_by_name('image:0'),
                           },
                    outputs={'global_descriptor': graph.get_tensor_by_name('global_descriptor:0')}
                )
                builder = tf.compat.v1.saved_model.Builder("./config/hfnet_v2_netvlad/")
                builder.add_meta_graph_and_variables(
                    sess=sess,
                    tags=[tf.compat.v1.saved_model.tag_constants.SERVING],
                    signature_def_map={'serving_default': signature}
                )
                builder.save()
                
class NetSuperPoint(object):

    def __init__(self, model_path):
        self.model_filepath = model_path
        self.load_graph(model_filepath=self.model_filepath)


    def load_graph(self, model_filepath):
        print("Loading model...")


        with tf.io.gfile.GFile(model_filepath, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())

        with tf.compat.v1.Session() as sess:
            with tf.Graph().as_default() as graph:
                tf.import_graph_def(graph_def, name='')
                signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
                    inputs={
                        'image':graph.get_tensor_by_name('image:0'),
                        'k':graph.get_tensor_by_name('k:0'),
                        'radius':graph.get_tensor_by_name('radius:0')
                           },
                    outputs={'keypoints': graph.get_tensor_by_name('keypoints:0'),
                        'local_descriptors': graph.get_tensor_by_name('local_descriptors:0')}
                )
                builder = tf.compat.v1.saved_model.Builder("./config/hfnet_v2_superpoint/")
                builder.add_meta_graph_and_variables(
                    sess=sess,
                    tags=[tf.compat.v1.saved_model.tag_constants.SERVING],
                    signature_def_map={'serving_default': signature}
                )
                builder.save()


Net(output_frozen_path)
#NetVLAD(output_frozen_path_netvlad)
#NetSuperPoint(output_frozen_path_sp)

Loading model...
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./config/hfnet_v2/saved_model.pb


In [ ]:
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS
conversion_params = conversion_params._replace(
    max_workspace_size_bytes=(1<<30))
conversion_params = conversion_params._replace(precision_mode="FP16")
conversion_params = conversion_params._replace(
    maximum_cached_engines=100)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir="./config/hfnet_v2_superpoint",
    conversion_params=conversion_params)
converter.convert()

In [ ]:
# np.shuffle()
def my_input_fn():
    for i in range(100):
        img = images[i]
        _img = np.expand_dims(img, axis=2)
        _img = np.array([_img]).astype(np.float32)
        #print(_img.shape)
    yield _img, 200, 4
converter.build(input_fn=my_input_fn)
converter.save("./config/hfnet_v2_superpoint_trt")